In [2]:
import numpy as np
from scipy.spatial.transform import Rotation
import re
import pandas as pd
#%cd ../..
from AutoCalibration import *

/home/ios/GitLab/rgbd-registration/rgb-lidar-reg


In [26]:
cal_stereo = np.load("./calib/multisense_seq_stereo_TR_icp_dist_to_new_improved_GT.npy")
cal_seq = np.load("calib/multisense_seq_27_RT_old_to_new_recentered_GT.npy")
cal_kitti = PyKitti2("/home/ios/data3/kitti/tracking/training/", "0000", with_labels="mask").calib.P_rect_20

                     
frame = []

repetitions = 5

stereo_0 = []
stereo_3 = []
stereo_5 = []

seq0 = []
seq3 = []
seq5 = []

#filename = "/home/ios/GitHub/automatic_lidar_camera_calibration/gNMI_results_std_new.txt"
#filename = "/home/ios/GitHub/automatic_lidar_camera_calibration/gNMI_results_stereo_005.txt"
#filename = "/home/ios/GitHub/automatic_lidar_camera_calibration/gNMI_results_stereo_007.txt"
#filename = "/home/ios/GitHub/automatic_lidar_camera_calibration/gNMI_results_stereo_0_2_6.txt"
filename = "/home/ios/GitHub/automatic_lidar_camera_calibration/gNMI_results_0_2_6.txt"

with open(filename, "r") as file:
    lines = []
    count = 0
    run = 0
    for line in file:
        if line.startswith("x"):
            count += 1
        
            if run == 0:
                stereo_0.append(line)
            elif run == 1:
                stereo_3.append(line)
            elif run == 2:
                stereo_5.append(line)
            elif run == 3:
                seq0.append(line)
            elif run == 4:
                seq3.append(line)
            elif run == 5:
                seq5.append(line)
            if count % repetitions == 0:
                run += 1
                 
stereo = [stereo_0, stereo_3, stereo_5]
seq = [seq0, seq3, seq5]
            
for name, seq, cal in zip(["Stereo", "Seq"], [stereo, seq], [cal_stereo, cal_seq]):

    true_R = cal[:3, :3]
    true_xyz = cal[:3, 3]
    true_y, true_p, true_r = Rotation.from_matrix(true_R).as_euler("zyx", degrees=True)
    
    for s, d in zip(stereo, [0, 3, 5]):
        mses = np.array([])
        angles = np.array([])
        for i in range(repetitions):
            x, y, z, roll, pitch, yaw = re.findall('[xyzrp]_?[deg]{0,3}:\s([-]{0,1}\d*.\d*)\[[mdeg]{1,3}\],?\s?', s[i]) 
            R_est = Rotation.from_euler("zyx", [yaw, pitch, roll], degrees=True).as_matrix()
            mses = np.append(mses, np.linalg.norm( true_xyz - np.asarray([x, y, z]).astype(np.float32)))
            angles = np.append(angles, quat_diff(true_R, R_est))
            
        mse_std = np.std(mses)
        mse_mean = np.mean(mses)
        mse_median = np.median(mses)

        angle_std = np.std(angles)
        angle_mean = np.mean(angles)
        angle_median = np.median(angles)

        frame.append([name, d, mse_std, mse_mean, mse_median, angle_std, angle_mean, angle_median])
        
frame = pd.DataFrame(frame, columns=["Name" , "Distortion[cm]", "MSE std", "MSE mean", "MSE median", "angle std", "angle mean", "angle median"])
frame
        

files 154


,Name,Distortion[cm],MSE std,MSE mean,MSE median,angle std,angle mean,angle median
0,Stereo,0,0.026652,0.073286,0.078613,0.725865,1.174448,0.671320
1,Stereo,3,0.031453,0.056806,0.051568,0.746404,1.087652,1.036639
2,Stereo,5,0.012065,0.056791,0.048835,0.387991,1.165849,1.321565
3,Seq,0,0.024446,0.072069,0.072090,0.635388,1.602684,1.201261
4,Seq,3,0.029107,0.055015,0.038693,0.608517,1.400491,1.297500
5,Seq,5,0.008991,0.063328,0.068539,0.316067,1.489513,1.464839


In [50]:
true_y, true_p, true_r

(88.17591584732864, 0.07559838260835568, 90.40399105866382)

In [51]:
yaw, pitch, roll

('89.016594', '-0.773366', '91.698709')

In [1]:
import sys
sys.path.insert(0, '../../datasets')
print(sys.path)
from PyKitti2Dataset import PyKitti2

['../../datasets', '', '/home/ios/anaconda3/envs/lccnet/lib/python36.zip', '/home/ios/anaconda3/envs/lccnet/lib/python3.6', '/home/ios/anaconda3/envs/lccnet/lib/python3.6/lib-dynload', '/home/ios/anaconda3/envs/lccnet/lib/python3.6/site-packages', '/home/ios/anaconda3/envs/lccnet/lib/python3.6/site-packages/correlation_cuda-0.0.0-py3.6-linux-x86_64.egg', '/home/ios/anaconda3/envs/lccnet/lib/python3.6/site-packages/IPython/extensions', '/home/ios/.ipython']
Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
cal_kitti = PyKitti2("/home/ios/data3/kitti/tracking/training/", "0010", with_labels="mask").calib.T_cam2_velo
                  
frame = []

repetitions = 5

kitti = []

filename = "/home/ios/GitHub/automatic_lidar_camera_calibration/gNMI_results_kitti.txt"

with open(filename, "r") as file:
    lines = []
    for line in file:
        if line.startswith("x"):
            kitti.append(line)
            
for name, seq, cal in zip(["KITTI"], [kitti], [cal_kitti]):
    true_R = cal[:3, :3]
    true_xyz = cal[:3, 3]
    true_y, true_p, true_r = Rotation.from_matrix(true_R).as_euler("zyx", degrees=True)
    
    mses = np.array([])
    angles = np.array([])
    for i in range(repetitions):
        x, y, z, roll, pitch, yaw = re.findall('[xyzrp]_?[deg]{0,3}:\s([-]{0,1}\d*.\d*)\[[mdeg]{1,3}\],?\s?', kitti[i]) 
        R_est = Rotation.from_euler("zyx", [yaw, pitch, roll], degrees=True).as_matrix()
        mses = np.append(mses, np.linalg.norm( true_xyz - np.asarray([x, y, z]).astype(np.float32)))
        angles = np.append(angles, quat_diff(true_R, R_est))

    mse_std = np.std(mses)
    mse_mean = np.mean(mses)
    mse_median = np.median(mses)


    angle_std = np.std(angles)
    angle_mean = np.mean(angles)
    angle_median = np.median(angles)

    frame.append([name, 6, mse_std, mse_mean, mse_median, angle_std, angle_mean, angle_median])
        
frame = pd.DataFrame(frame, columns=["Name" , "Distortion[cm]", "MSE std", "MSE mean", "MSE median", "angle std", "angle mean", "angle median"])
frame

files 294


,Name,Distortion[cm],MSE std,MSE mean,MSE median,angle std,angle mean,angle median
0,KITTI,6,0.047937,0.084811,0.088513,5.691025,8.170018,11.393538


In [12]:
%pwd

'/home/ios/GitLab/rgbd-registration/rgb-lidar-reg/baselines/gNMI'